# Modelo SVC para detección de noticias falsas

En este notebook, entrenaremos un modelo de clasificación basado en **Support Vector Classification (SVC)** para predecir si una afirmación política es **verdadera** o **falsa**.

Se utilizarán técnicas de preprocesamiento de texto (TF-IDF) y metadatos categóricos (codificados y escalados). Además, se aplicará balanceo de clases con SMOTE, selección de características, y búsqueda de hiperparámetros con GridSearchCV.


In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.kernel_approximation import RBFSampler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

### Carga dataset

Se cargan los datasets de entrenamiento y prueba preprocesados, que contienen:

- `statement`: texto de la afirmación (columna principal para NLP).
- `subject`: tema de la afirmación (categórica).
- `speaker`: persona que hizo la afirmación (categórica).
- `party_affiliation`: partido político (categórica).
- `state_info_*`: columnas relacionadas con la ubicación geográfica.
- `label`: etiqueta binaria (0 = falso, 1 = verdadero) para entrenamiento.

In [3]:
# Cargar los datasets limpios
train_data = pd.read_csv('C:/Users/inesg/dev/LBBYs_CH2/data/processed/train_simp_preprocess_v2.csv')
test_data = pd.read_csv('C:/Users/inesg/dev/LBBYs_CH2/data/processed/test_simp_preprocess_v2.csv')

### Constantes

In [6]:
submissions_folder = "C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision"


### Preprocesamiento de los datos
#### 1. Preprocesamiento del texto con TF-IDF
Se transforma la columna `statement` con TF-IDF, limitando a las 1000 palabras más importantes y eliminando stopwords en inglés.

Esto convierte el texto en vectores numéricos que el modelo puede procesar.

In [7]:
# Crear y entrenar el tokenizer con TF-IDF
tokenizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = tokenizer.fit_transform(train_data['statement']).toarray()

#### 2. Preprocesamiento de metadatos
Se codifican las variables categóricas (`subject`, `speaker`, `party_affiliation`) con LabelEncoder y se reduce la información de `state_info_*` a un indicador binario de presencia.

In [8]:
label_encoder_subject = LabelEncoder()
label_encoder_speaker = LabelEncoder()
label_encoder_party = LabelEncoder()

train_data['subject_encoded'] = label_encoder_subject.fit_transform(train_data['subject'])
train_data['speaker_encoded'] = label_encoder_speaker.fit_transform(train_data['speaker'])

state_info_columns = [col for col in train_data.columns if col.startswith('state_info')]
train_data['state_info_encoded'] = train_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

train_data['party_affiliation_encoded'] = label_encoder_party.fit_transform(train_data['party_affiliation'])

X_metadata = train_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

#### 3. Escalar los metadatos
Se normalizan los valores numéricos de los metadatos para que estén en la misma escala y el modelo no se sesgue.

In [9]:
scaler = StandardScaler()
X_metadata_scaled = scaler.fit_transform(X_metadata)

#### 4. Unión texto y metadatos
Se concatenan las características textuales y los metadatos para formar la matriz de características completa.

In [10]:
X_final = np.concatenate([X_text, X_metadata_scaled], axis=1)
y = train_data['label'].values

### División en train/test
Dividimos los datos en conjunto de entrenamiento y prueba para evaluar la generalización.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

### Manejo clases desbalanceadas
Si las clases están desbalanceadas, se aplica `SMOTE` para generar muestras sintéticas de la clase minoritaria solo en el conjunto de entrenamiento.

In [12]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(f"Clases balanceadas en train tras SMOTE: {np.bincount(y_resampled)}")

Clases balanceadas en train tras SMOTE: [4616 4616]


### Entrenamiento

#### 1. GridSearchCV
Se busca la mejor combinación de `C`, `gamma` y `kernel` para el modelo SVC, usando validación cruzada y priorizando el recall de la clase 0 (falsas).

(En primer lugar he aplicado el gridSearch priorizando el accuaracy, luego lo he modificado para priorizar el f1-score de la clase 0 que era el  que peor resultados tenia)

In [37]:
from sklearn.metrics import make_scorer, recall_score

In [48]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['linear', 'rbf']
}

svc = SVC(class_weight='balanced')
# Usar recall de clase 0
scorer_recall_0 = make_scorer(recall_score, pos_label=0)
grid_search = GridSearchCV(svc, param_grid, scoring=scorer_recall_0, cv=5, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

Mejores parámetros encontrados: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}


##### Evaluación

In [49]:
# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Realizar predicciones
y_pred_test = best_model.predict(X_test)

print("Métricas en test:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_test):.4f}")
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))

Métricas en test:
Accuracy: 0.6291
Precision: 0.6799
Recall: 0.8253
F1-Score: 0.7456
              precision    recall  f1-score   support

           0       0.43      0.25      0.32       611
           1       0.68      0.83      0.75      1179

    accuracy                           0.63      1790
   macro avg       0.55      0.54      0.53      1790
weighted avg       0.59      0.63      0.60      1790

[[153 458]
 [206 973]]


##### Submission

In [41]:
def encode_with_unknown_handling(le, series):
    # Valores conocidos en train
    known_labels = set(le.classes_)
    # Reemplazar valores no conocidos por el más frecuente o un valor fijo
    replacement = le.classes_[0]  # O el más frecuente en train
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

# Aplicar a cada columna categórica
test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])
test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test
test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar todo para obtener la matriz final con 1004 features
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Predecir usando el modelo entrenado y matriz final
y_pred_submission = best_model.predict(X_test_final)

# Crear DataFrame submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_submission
})

submission_csv_path = f"{submissions_folder}/submission_grid_search_updated_1.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")

Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_grid_search_updated_1.csv


#### 2. SelectKBest
Esta técnica selecciona las características más relevantes para la clasificación basándose en el análisis estadístico **ANOVA F-value**.

Seleccionar solo las características más importantes puede ayudar a reducir ruido, mejorar el rendimiento y acelerar el entrenamiento.

In [22]:
selector = SelectKBest(f_classif, k=100)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)


c:\Users\inesg\dev\LBBYs_CH2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [1002] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\inesg\dev\LBBYs_CH2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


##### Evaluación

In [23]:
svc_selected_model = SVC(C=10, gamma=1, kernel='rbf')
svc_selected_model.fit(X_train_selected, y_resampled)

y_pred_selectKBest = svc_selected_model.predict(X_test_selected)

print("Métricas con SelectKBest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_selectKBest):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selectKBest):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selectKBest):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_selectKBest):.4f}")
print(classification_report(y_test, y_pred_selectKBest))
print(confusion_matrix(y_test, y_pred_selectKBest))

Métricas con SelectKBest:
Accuracy: 0.5642
Precision: 0.7021
Recall: 0.5878
F1-Score: 0.6399
              precision    recall  f1-score   support

           0       0.39      0.52      0.45       611
           1       0.70      0.59      0.64      1179

    accuracy                           0.56      1790
   macro avg       0.55      0.55      0.54      1790
weighted avg       0.60      0.56      0.57      1790

[[317 294]
 [486 693]]


##### Submission

In [24]:
# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test con manejo de valores desconocidos
def encode_with_unknown_handling(le, series):
    known_labels = set(le.classes_)
    replacement = le.classes_[0]
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])

test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar texto y metadatos
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Aplicar SelectKBest al test
X_test_selected = selector.transform(X_test_final)

# Predecir con modelo entrenado
y_pred_selectKBest_submission = svc_selected_model.predict(X_test_selected)

# Crear DataFrame de submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_selectKBest_submission
})

# Guardar CSV
submission_csv_path = f"{submissions_folder}/submission_selectKBest.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")


Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_selectKBest.csv


#### 3. RFE (Recursive Feature Elimination)

RFE es una técnica que elimina recursivamente las características menos importantes basándose en la importancia que asigna un estimador (en este caso un SVC lineal).

Se entrena un modelo con todas las características, se elimina la menos importante, y se repite hasta quedarse con el número deseado de características.

Esto ayuda a obtener un subconjunto de características muy relevantes para mejorar la generalización y reducir ruido.

In [25]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [26]:
svc_base = SVC(kernel='linear', C=1)
rfe_selector = RFE(estimator=svc_base, n_features_to_select=50)

X_train_rfe = rfe_selector.fit_transform(X_resampled, y_resampled)

X_test_rfe = rfe_selector.transform(X_test)


##### Evaluación

In [27]:
svc_rfe_model = SVC(C=10, gamma=1, kernel='rbf')
svc_rfe_model.fit(X_train_rfe, y_resampled)

y_pred_rfe = svc_rfe_model.predict(X_test_rfe)

print(f"Accuracy con RFE: {accuracy_score(y_test, y_pred_rfe):.4f}")
print(f"Precision con RFE: {precision_score(y_test, y_pred_rfe):.4f}")
print(f"Recall con RFE: {recall_score(y_test, y_pred_rfe):.4f}")
print(f"F1-Score con RFE: {f1_score(y_test, y_pred_rfe):.4f}")

print("Reporte de clasificación con RFE:")
print(classification_report(y_test, y_pred_rfe))
print("Matriz de confusión con RFE:")
print(confusion_matrix(y_test, y_pred_rfe))

Accuracy con RFE: 0.5056
Precision con RFE: 0.7466
Recall con RFE: 0.3774
F1-Score con RFE: 0.5014
Reporte de clasificación con RFE:
              precision    recall  f1-score   support

           0       0.39      0.75      0.51       611
           1       0.75      0.38      0.50      1179

    accuracy                           0.51      1790
   macro avg       0.57      0.57      0.51      1790
weighted avg       0.62      0.51      0.50      1790

Matriz de confusión con RFE:
[[460 151]
 [734 445]]


##### Submission

In [28]:
# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test con manejo de valores desconocidos
def encode_with_unknown_handling(le, series):
    known_labels = set(le.classes_)
    replacement = le.classes_[0]
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])

test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar texto y metadatos
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Aplicar RFE selector al test
X_test_rfe = rfe_selector.transform(X_test_final)

# Predecir con modelo entrenado
y_pred_rfe_submission = svc_rfe_model.predict(X_test_rfe)

# Crear DataFrame de submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_rfe_submission
})

# Guardar CSV
submission_csv_path = f"{submissions_folder}/submission_rfe.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")


Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_rfe.csv


#### 4. SVC sobre SBERT Embeddings con GridSearchCV
Para abordarlo, primero convertimos cada afirmación de texto en un vector denso usando un modelo SBERT ligero (`all-MiniLM-L6-v2`), lo que nos permite capturar relaciones semánticas y contexto más allá de las simples frecuencias de palabra. A continuación, normalizamos estos vectores con un escalador estándar y entrenamos un clasificador SVC con kernel RBF y pesos de clase equilibrados para enfrentar el desbalance de etiquetas. Para ajustar los hiperparámetros críticos (`C` y `gamma`), aplicamos una búsqueda de cuadrícula (`GridSearchCV`) focalizada en maximizar la métrica macro-F1 mediante validación cruzada estratificada de tres pliegues. Una vez seleccionados los valores óptimos, evaluamos el rendimiento en un conjunto de validación independiente, reentrenamos el modelo final con todo el conjunto de entrenamiento y generamos las predicciones para el test, guardándolas en el CSV de envío.  

In [64]:
# 1. Carga datos de entrenamiento
train_df = pd.read_csv('train_simp_preprocess_v2.csv')
X = train_df['statement'].tolist()
y = train_df['label'].values

# 2. Split train/test para búsqueda de hiperparámetros
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Define embedder SBERT ligero
enbedder = SentenceTransformer('all-MiniLM-L6-v2')

def embed_corpus(texts):
    return enbedder.encode(texts, show_progress_bar=True)

# 4. Pipeline: embeddings → escalado → SVC
pipe = Pipeline([
    ('embed',  FunctionTransformer(lambda txts: embed_corpus(txts), validate=False)),
    ('scale',  StandardScaler()),
    ('svc',    SVC(kernel='rbf', class_weight='balanced'))
])

# 5. GridSearchCV sobre f1_macro para C y gamma
param_grid = {
    'svc__C':     [0.1, 1, 10],
    'svc__gamma': ['scale', 0.01, 0.1]
}
grid = GridSearchCV(pipe, param_grid, cv=3, scoring='f1_macro', n_jobs=-1, verbose=2)

# 6. Entrenar grid
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)

# 7. Evaluación en validación
y_val_pred = grid.predict(X_val)
print(classification_report(y_val, y_val_pred, digits=4))

# 8. Retrain mejor modelo con todos los datos de entrenamiento
best_pipe = grid.best_estimator_
best_pipe.fit(X, y)

# 9. Generar submission
test_df = pd.read_csv('test_simp_preprocess_v2.csv')
X_test = test_df['statement'].tolist()
preds = best_pipe.predict(X_test)
submission = pd.DataFrame({'id': test_df['id'], 'label': preds})
submission.to_csv('submission.csv', index=False)
print("Submission guardada en submission.csv")

Fitting 3 folds for each of 9 candidates, totalling 27 fits


Batches: 100%|██████████| 224/224 [00:18<00:00, 12.37it/s]


Best params: {'svc__C': 1, 'svc__gamma': 'scale'}


Batches: 100%|██████████| 56/56 [00:04<00:00, 13.97it/s]


              precision    recall  f1-score   support

           0     0.4714    0.5626    0.5130       631
           1     0.7338    0.6566    0.6931      1159

    accuracy                         0.6235      1790
   macro avg     0.6026    0.6096    0.6030      1790
weighted avg     0.6413    0.6235    0.6296      1790



Batches: 100%|██████████| 120/120 [00:08<00:00, 13.34it/s]


Submission guardada en submission.csv


## Conclusiones

1. **Macro-F1 Score**  
   - El mejor macro-F1 se obtiene con el **SVC sobre SBERT Embeddings** (0.6030), muy por encima de **SelectKBest** (0.54) y **RFE** (0.51).

2. **Balance entre clases**  
   - Aunque RFE logró la mayor precisión macro (0.57), su macro-F1 cae a 0.51 porque sacrifica recall en la clase mayoritaria (“true”).  
   - El SVC sobre SBERT mejora notablemente el F1 de ambas clases (0.5130 para “fake” y 0.6931 para “true”), ofreciendo un trade-off más equilibrado.

3. **Por qué SVC puro con TF-IDF se queda corto**  
   - **Alta dimensionalidad y sparsity** de TF-IDF penaliza la generalización.  
   - **Falta de contexto**: no captura relaciones sintácticas ni semánticas profundas.  
   - **Desbalance de clases**: con macro-F1 alrededor de 0.5, el modelo no aprende bien ninguno de los dos segmentos.

4. **Vías de mejora**  
   - Integrar **embeddings pre-entrenados** (SBERT, FastText, BERT) como base, tal como hicimos en el SVC sobre SBERT.  
   - Aplicar **reduction of dimensionality** (SVD, UMAP) o **kernel approximation** antes de SVC.  
   - Probar **ensembles suaves** (votación, stacking) y **calibración de probabilidades** para afinar scores.  
   - Explorar **modelos basados en transformadores** o ensamblajes avanzados (Random Forest, XGBoost, LightGBM) sobre embeddings enriquecidos.  
